In [1]:
import json
import pandas as pd


# Load the data
- what is pandas
- different data sources

In [ ]:

#load the json file with pandas library into a data frame

data = pd.read_json('products.json')



# Clean the data
- average rating is incorrect
- set the correct average data

# Exploratory Data Analysis
- What items have the most reviews?
- What is the average rating of items that have the most reviews?
- Correlation between available color and ratings?
- Correlation between ar compatability and ratings?
- Correlation between product type and ratings?

# Visualize the data

# Create a Model
- What type, color would likely get the most reviews?